## Colab用 環境準備

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Mecabインストール
!sudo apt install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file swig
#形態素解析器辞書としてmecab-ipadic-NEologd
!sudo apt install git make curl xz-utils file
%cd /tmp
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
%cd mecab-ipadic-neologd
!./bin/install-mecab-ipadic-neologd -n
# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc
!pip install mojimoji
!pip install mecab-python3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
make set to manually installed.
xz-utils is already the newest version (5.2.2-1.3).
xz-utils set to manually installed.
curl is already the newest version (7.58.0-2ubuntu3.10).
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following additional packages will be installed:
  libmagic-mgc libmagic1 libmecab2 mecab-ipadic mecab-jumandic
  mecab-jumandic-utf8 mecab-utils swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  file libmagic-mgc libmagic1 libmecab-dev libmecab2 mecab mecab-ipadic
  mecab-ipadic-utf8 mecab-jumandic mecab-jumandic-utf8 mecab-utils swig
  swig3.0
0 upgraded, 13 newly installed, 0 to remove and 21 not upgraded.
Need to get 30.4 MB of archives.
After this operation, 288 MB of additional disk space will be used.
Get:1 http://archiv

In [3]:
import os
import re
import time
from contextlib import contextmanager
from operator import itemgetter

import numpy as np
import pandas as pd
#最大表示行数の指定（ここでは50行を指定）
pd.set_option('display.max_rows', 200)
#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 200)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer as Tfidf
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet


import re, MeCab
from glob import glob
import mojimoji
import warnings
warnings.simplefilter("ignore")

from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU
from keras.layers.core import Activation, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model
from keras.utils import np_utils

In [4]:
import os
import urllib.request

def download_stopwords(path):
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    if os.path.exists(path):
        print('File already exists.')
    else:
        print('Downloading...')
        # Download the file from `url` and save it locally under `file_name`:
        urllib.request.urlretrieve(url, path)

def create_stopwords(file_path):
    stop_words = []
    for w in open(path, "r"):
        w = w.replace('\n','')
        if len(w) > 0:
          stop_words.append(w)
    return stop_words    

path = "stop_words.txt"
download_stopwords(path)
stop_words = create_stopwords(path)

Downloading...


In [5]:
tagger = MeCab.Tagger("-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
def make_wakati(sentence):
    # MeCabで分かち書き
    sentence = sentence.lower()
    sentence = tagger.parse(sentence)
    sentence = mojimoji.zen_to_han(sentence)
    # 記号もろもろ除去
    sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—♬◉ᴗ͈ˬ●▲★☆⭐️⭕⚡⚠①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮♡⭐︎〇◎◆◼♦▼◇△□(：〜～＋=)／*&^%$#@!~`)♪ᴖ◡ᴖｰ{}［］↑↓←→➡⇩™･⊡…\[\]\"\'\”\’:;<>?＜＞〔〕\r＼−〈〉？、､。｡・,\./『』【】｢｣「」→←○《》≪≫\n\u3000⭕]+', "", sentence)
    # 絵文字除去
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           "]+", flags=re.UNICODE)
    sentence = emoji_pattern.sub(r'', sentence)
    return sentence

def title_torkenize(sentence):
    sentence = mojimoji.zen_to_han(sentence)
    sentence = re.sub("[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒♬◉ᴗ͈ˬ—●▲★☆⭐️⭕⚡⚠①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮♡⭐︎〇◎◆♦▼◼◇△□(：〜～＋=)／*&^%$#@!~`)♪ᴖ◡ᴖｰ{}［］↑↓←→➡⇩™･⊡…\[\]\"\'\”\’:;<>?＜＞〔〕\r＼−〈〉？、､。｡・,\./『』【】｢｣「」→←○《》≪≫\n\u3000]", " ", sentence)
    sentence = re.sub("[あ-ん]", " ", sentence)
    sentence = re.sub("( |　)+", " ", sentence)
    sentence = sentence.lower()
    #〇〇様専用を除く
    sentence = re.sub("[^ ]*専用", "", sentence)
    sentence = re.sub("[^ ]*様", "", sentence)
    #1文字のアルファベットを除く
    sentence = re.sub(" [a-z]{1}[^(a-z)]", " ", sentence)
    # 絵文字除去
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           "]+", flags=re.UNICODE)
    sentence = emoji_pattern.sub(r'', sentence)
    sentence = sentence.strip()

    return sentence

def preprocess(df):
    df["price"] = df["price"].str.replace(r"\D", "").astype(np.float)
    
    #列ズレを修正
    pattern = re.compile(r"^(?!.*(傷や汚れあり|全体的に状態が悪い|やや傷や汚れあり|未使用に近い|目立った傷や汚れなし|新品、未使用)).+$")
    invalid = df["status"].str.match(pattern)

    df.loc[invalid, "description"] = df.loc[invalid, "description"] + "\n" + df.loc[invalid, "status"]
    df.loc[invalid, "status"]      = df.loc[invalid, "shipping"]
    df.loc[invalid, "shipping"]    = df.loc[invalid, "method"]
    df.loc[invalid, "method"]      = df.loc[invalid, "region"]
    df.loc[invalid, "period"]      = "未定"
    
    df["title"] = df["title"] + " " + df["sub_category_1"] + " " + df["sub_category_2"] + " " + df["brand"]

    df = df.drop(columns=["sub_category_1", "sub_category_2", "brand"])
    
    status_dict = {'新品、未使用': "best",
                   '未使用に近い': "Very Good",
                   '目立った傷や汚れなし': "good",
                   '傷や汚れあり': "Poor",
                   'やや傷や汚れあり': "very poor",
                   '全体的に状態が悪い': "worst"
                  }
    
    #配送負担をラベルエンコーディング
    shipping_dict = {'送料込み(出品者負担)': 0, '着払い(購入者負担)': 1}

    df["status"] = df["status"].map(status_dict)
    df["shipping"] = df["shipping"].map(shipping_dict)
    
    #トークナイズ
    df["title"] = df["title"].apply(title_torkenize)
    df["description"] = df["description"].apply(make_wakati)
    
    #不要列削除
    df = df.drop(columns=["url", "seller", "rating", "method", "region", "period", "recent_comment", "timestamp"])
    return df

In [6]:
cd /

/


In [7]:
cd content/drive/My Drive/pj_horidasimono/code-analysis

/content/drive/My Drive/pj_horidasimono/code-analysis


In [8]:
cd ../code/

/content/drive/My Drive/pj_horidasimono/code


In [9]:
from model_ridge import ModelRidge
from model_nn import ModelMLP
from model_lgb import ModelLightGBM

## モデル学習

In [10]:
from load_data import load_data_from_gcs
#df_ = load_data_from_gcs()
df_ = pd.read_pickle("../dataset/df1026.pickle")
#df_.to_pickle("../dataset/df1026.pickle")

In [22]:
base_params = {
                            'input_dropout': 0.2,
                            'hidden_layers': 3,
                            'hidden_units': 256,
                            'hidden_activation': 'relu',
                            'hidden_dropout': 0.2,
                            'batch_norm': 'before_act',
                            'optimizer': {'type': 'adam', 'lr': 5e-5},
                            'batch_size': 64,
                          }

lgb_params = {
                          'learning_rate': 0.1,
                          'objective': 'regression',
                          'max_depth': 7,
                          'min_child_samples': 10,
                          'num_leaves': 76,
                          'lambda_l1': 1.1240179595393505e-08,
                          'lambda_l2': 1.2102177820339625e-08,
                          'verbosity': 0,
                          'metric': 'RMSE',
                          'feature_fraction': 0.8,
                          'nthread': 4,
                          'bagging_freq': 6,
                          'bagging_fraction': 0.9869425400924351,
                          'num_round': 10000
                        }

def regression_metrics(y_true, y_pred):
  
  def mean_absolute_percentage_error(y_true, y_pred): 
      return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
      
  mae   =  mean_absolute_error(y_true, y_pred)
  try:
    rmsle =  np.sqrt(mean_squared_log_error(y_true, y_pred))
  except:
    rmsle =  -1

  mse   =  mean_squared_error(y_true, y_pred)
  rmse  =  np.sqrt(mean_squared_error(y_true, y_pred))
  mape  =  mean_absolute_percentage_error(y_true, y_pred)
  r2    =  r2_score(y_true, y_pred)

  scores = pd.DataFrame(data={"MAE": mae, "MSE": mse, "RMSE": rmse, "RMSLE": rmsle, "MAPE": mape, "R2": r2}, index=["score"])
  return scores


In [40]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')
    
def on_field(f: str, *vec):
    return make_pipeline(FunctionTransformer(itemgetter(f), validate=False), *vec)

def to_records(df: pd.DataFrame):
    return df.to_dict(orient='records')

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

vectorizer = make_union(
            on_field("title", Tfidf(stop_words=stop_words, max_features=300000, token_pattern="(?u)\\b\\w\\w+\\b")),
            on_field("description", Tfidf(stop_words=stop_words, max_features=500000, token_pattern="(?u)\\b\\w\\w+\\b", ngram_range=(1, 2))),
            on_field(['shipping', 'status'],
                 FunctionTransformer(to_records, validate=False), DictVectorizer()))

with timer("preprocess"):
    df = preprocess(df_.copy())
    X = df.drop(columns="price")
    y = df["price"]
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)
    vectorizer.fit(pd.concat([X_train, X_valid]))
    X_all   = vectorizer.transform(pd.concat([X_train, X_valid])).astype(np.float32)
    X_train = vectorizer.transform(X_train).astype(np.float32)
    X_valid = vectorizer.transform(X_valid).astype(np.float32)
    del X, y
 


[preprocess] done in 154 s


In [ ]:
from sklearn.decomposition import TruncatedSVD

with timer("svd"):
  svd = TruncatedSVD(n_components=10000)
  svd.fit(X_all)
  train_svd = svd.transform(X_train)
  test_svd = svd.transform(X_valid)

In [1]:
with timer("fit and predict"):
    model = ModelMLP(base_params)
    model.fit(train_svd, y_train, test_svd, y_valid)
    pred = model.predict(test_svd)


NameError: ignored

In [33]:
display(regression_metrics(y_valid, pred))

,MAE,MSE,RMSE,RMSLE,MAPE,R2
score,2432.54747,2.044119e+07,4521.19376,0.438182,31.57295,0.630396


## アンサンブル

In [ ]:
import pickle
# with open("../dataset/X_train.pkl", "wb") as f:
#   pickle.dump(X_train, f)

# with open("../dataset/X_valid.pkl", "wb") as f:
#   pickle.dump(X_valid, f)

# with open("../dataset/y_train.pkl", "wb") as f:
#   pickle.dump(y_train, f)

# with open("../dataset/y_valid.pkl", "wb") as f:
#   pickle.dump(y_valid, f)

with open("../dataset/X_train.pkl", "rb") as f:
  X_train = pickle.load(f)

with open("../dataset/X_valid.pkl", "rb") as f:
  X_valid = pickle.load(f)
  
with open("../dataset/y_train.pkl", "rb") as f:
  y_train = pickle.load(f)
  
with open("../dataset/y_valid.pkl", "rb") as f:
  y_valid = pickle.load(f)

In [ ]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=3, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x[tr_idx], train_x[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)

        #model.run_name = model.run_name + "_fold_" + str(i)
        #model.save_model()

        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

# 1層目のモデル
model_1a = ModelLightGBM(lgb_params)
pred_train_1a, pred_test_1a = predict_cv(model_1a, X_train, y_train, X_valid)

model_1b = ModelMLP(base_params)
pred_train_1b, pred_test_1b = predict_cv(model_1b, X_train, y_train, X_valid)

model_1c = ModelRidge()
pred_train_1c, pred_test_1c = predict_cv(model_1c, X_train, y_train, X_valid)

# 1層目のモデルの評価
display(regression_metrics(y_train, pred_train_1a))
display(regression_metrics(y_train, pred_train_1b))
# display(regression_metrics(y_train, pred_train_1c))
# 予測値を特徴量としてデータフレームを作成
train_x_2 = pd.DataFrame({'pred_1a': pred_train_1a, 'pred_1b': pred_train_1b, 'pred_1c': pred_train_1c}).values
test_x_2  = pd.DataFrame({'pred_1a': pred_test_1a,  'pred_1b': pred_test_1b, 'pred_1c': pred_test_1c}).values

# 2層目のモデル
# pred_train_2は、2層目のモデルの学習データのクロスバリデーションでの予測値
# pred_test_2は、2層目のモデルのテストデータの予測値
x_scaler = StandardScaler()
train_x_2 = x_scaler.fit_transform(train_x_2)
test_x_2  = x_scaler.transform(test_x_2)

model_2 = LinearRegression()
model_2.fit(train_x_2, y_train)
y_pred = model_2.predict(test_x_2)

display(regression_metrics(y_valid, y_pred))

In [ ]:
display(regression_metrics(y_train, pred_train_1a))
display(regression_metrics(y_train, pred_train_1b))
display(regression_metrics(y_train, pred_train_1c))
display(regression_metrics(y_valid, y_pred))

,MAE,MSE,RMSE,RMSLE,MAPE,R2
score,3212.799402,2.528604e+07,5028.522594,-1,53.337337,0.537519


,MAE,MSE,RMSE,RMSLE,MAPE,R2
score,2519.463324,2.115521e+07,4599.479434,0.454735,33.334483,0.613072


,MAE,MSE,RMSE,RMSLE,MAPE,R2
score,2855.902809,2.473408e+07,4973.336957,0.463165,37.092993,0.547615


,MAE,MSE,RMSE,RMSLE,MAPE,R2
score,2539.491938,1.929924e+07,4393.090088,0.444133,39.190094,0.651044


In [ ]:

# 1層目のモデルの評価
display(regression_metrics(y_train, pred_train_1a))
display(regression_metrics(y_train, pred_train_1b))
display(regression_metrics(y_train, pred_train_1c))
# 予測値を特徴量としてデータフレームを作成
train_x_2 = pd.DataFrame({'pred_1a': pred_train_1a, 'pred_1b': pred_train_1b, "pred_1c": pred_train_1c}).values
test_x_2  = pd.DataFrame({'pred_1a': pred_test_1a,  'pred_1b': pred_test_1b,  "pred_1c": pred_test_1c}).values

# 2層目のモデル
# pred_train_2は、2層目のモデルの学習データのクロスバリデーションでの予測値
# pred_test_2は、2層目のモデルのテストデータの予測値
x_scaler = StandardScaler()
train_x_2 = x_scaler.fit_transform(train_x_2)
test_x_2  = x_scaler.transform(test_x_2)

model_2 = LinearRegression()
model_2.fit(train_x_2, y_train)
y_pred = model_2.predict(test_x_2)

display(regression_metrics(y_valid, y_pred))

In [ ]:

lgb_params = {
                          'learning_rate': 0.1,
                          'objective': 'regression',
                          'verbosity': 0,
                          'metric': 'RMSE',
                          'num_round': 10000
                        }


model = ModelLightGBM(lgb_params)
model.fit(X_train, y_train, X_valid, y_valid)

[1]	train's rmse: 7318.59	valid's rmse: 7471.94
Training until validation scores don't improve for 20 rounds.
[2]	train's rmse: 7204.59	valid's rmse: 7358.98
[3]	train's rmse: 7109.91	valid's rmse: 7263.83
[4]	train's rmse: 7023.68	valid's rmse: 7176.94
[5]	train's rmse: 6951.04	valid's rmse: 7105.76
[6]	train's rmse: 6883.7	valid's rmse: 7042.07
[7]	train's rmse: 6823.93	valid's rmse: 6984.83
[8]	train's rmse: 6770.08	valid's rmse: 6931.8
[9]	train's rmse: 6705.71	valid's rmse: 6871.55
[10]	train's rmse: 6661.15	valid's rmse: 6829.71
[11]	train's rmse: 6616.4	valid's rmse: 6785.26
[12]	train's rmse: 6568.43	valid's rmse: 6739.29
[13]	train's rmse: 6534.35	valid's rmse: 6709.32
[14]	train's rmse: 6496.17	valid's rmse: 6673.53
[15]	train's rmse: 6461.29	valid's rmse: 6643.06
[16]	train's rmse: 6427.2	valid's rmse: 6611.27
[17]	train's rmse: 6398.48	valid's rmse: 6584.97
[18]	train's rmse: 6367.92	valid's rmse: 6556.67
[19]	train's rmse: 6340.91	valid's rmse: 6531.1
[20]	train's rmse: 63

KeyboardInterrupt: ignored

In [ ]:
a[22120]

array([ 2000.        , 31108.13232186])

In [ ]:
y_valid.iloc[22120]

2000.0

In [ ]:
pd.read_pickle

url                  https://www.mercari.com/jp/items/m92582569663/
title                     ⭐︎SanDisk SSD 2.5インチ/ Ultra 3D 1TB【新品未開封】
seller                                                      ヨグソトゥース
rating                                                         56|0
price                                                       ¥10,000
sub_category_1                                             PC/タブレット
sub_category_2                                                PCパーツ
brand                                                              
status                                                       新品、未使用
shipping                                                送料込み(出品者負担)
method                                                    らくらくメルカリ便
region                                                          東京都
period                                                      1~2日で発送
description       未開封の新品になります。\n\n申し訳ございませんが値下げ交渉には応じかねますので\n表示さ...
recent_comment                                  

In [ ]:
pd.read_pickle("../dataset/df1022.pickle").iloc[22120]

url                  https://www.mercari.com/jp/items/m92582569663/
title                     ⭐︎SanDisk SSD 2.5インチ/ Ultra 3D 1TB【新品未開封】
seller                                                      ヨグソトゥース
rating                                                         56|0
price                                                       ¥10,000
sub_category_1                                             PC/タブレット
sub_category_2                                                PCパーツ
brand                                                              
status                                                       新品、未使用
shipping                                                送料込み(出品者負担)
method                                                    らくらくメルカリ便
region                                                          東京都
period                                                      1~2日で発送
description       未開封の新品になります。\n\n申し訳ございませんが値下げ交渉には応じかねますので\n表示さ...
recent_comment                                  

In [ ]:
from optuna.integration import lightgbm as lgb

dtrain = lgb.Dataset(X_train[:50000], label=y_train[:50000])
eval_data = lgb.Dataset(X_valid, label=y_valid)

param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
    }

best_params = {}
tuning_history = []

best = lgb.train(param, 
                dtrain,
                valid_sets=eval_data,
                num_boost_round=500,
                verbose_eval=50,
                early_stopping_rounds=10)


"""
###実際にチューニングしてくれているパラメータ###

param = {
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
"""

[I 2020-10-26 14:27:35,428] A new study created in memory with name: no-name-0e227e05-4aeb-46ed-ac38-059773d6e66d


  0%|          | 0/7 [00:00<?, ?it/s]

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 10 rounds.
[50]	valid_0's rmse: 6107.33
[100]	valid_0's rmse: 5851.42
[150]	valid_0's rmse: 5738.6
[200]	valid_0's rmse: 5670.77
[250]	valid_0's rmse: 5628.65
[300]	valid_0's rmse: 5596.73
[350]	valid_0's rmse: 5572.66
[400]	valid_0's rmse: 5554.95
[450]	valid_0's rmse: 5541.07
[500]	valid_0's rmse: 5531.92
Did not meet early stopping. Best iteration is:
[499]	valid_0's rmse: 5531.92




feature_fraction, val_score: 5531.919080:   0%|          | 0/7 [04:56<?, ?it/s]

feature_fraction, val_score: 5531.919080:  14%|#4        | 1/7 [04:56<29:38, 296.49s/it][I 2020-10-26 14:32:31,931] Trial 0 finished with value: 5531.919079908854 and parameters: {'feature_fraction': 0.8}. Best is trial 0 with value: 5531.919079908854.


feature_fraction, val_score: 5531.919080:  14%|#4        | 1/7 [04:56<29:38, 296.49s/it]

Training until validation scores don't improve for 10 rounds.
[50]	valid_0's rmse: 6114.42
[100]	valid_0's rmse: 5868.84
[150]	valid_0's rmse: 5749.17
[200]	valid_0's rmse: 5684.47
[250]	valid_0's rmse: 5641.99
[300]	valid_0's rmse: 5610.97
[350]	valid_0's rmse: 5591.88
[400]	valid_0's rmse: 5573.19
[450]	valid_0's rmse: 5559.79
[500]	valid_0's rmse: 5546.9
Did not meet early stopping. Best iteration is:
[500]	valid_0's rmse: 5546.9




feature_fraction, val_score: 5531.919080:  14%|#4        | 1/7 [10:01<29:38, 296.49s/it]

feature_fraction, val_score: 5531.919080:  29%|##8       | 2/7 [10:01<24:54, 298.90s/it][I 2020-10-26 14:37:36,518] Trial 1 finished with value: 5546.899304352368 and parameters: {'feature_fraction': 0.8999999999999999}. Best is trial 0 with value: 5531.919079908854.


feature_fraction, val_score: 5531.919080:  29%|##8       | 2/7 [10:01<24:54, 298.90s/it]

...

Training until validation scores don't improve for 10 rounds.
[50]	valid_0's rmse: 5880.48
[100]	valid_0's rmse: 5661.46
[150]	valid_0's rmse: 5579.28
[200]	valid_0's rmse: 5537.86
[250]	valid_0's rmse: 5516.62
[300]	valid_0's rmse: 5504.91
[350]	valid_0's rmse: 5497.26
[400]	valid_0's rmse: 5491.41
[450]	valid_0's rmse: 5488.62
Early stopping, best iteration is:
[484]	valid_0's rmse: 5486.89




min_data_in_leaf, val_score: 5486.889965:  60%|######    | 3/5 [27:30<11:53, 356.75s/it]

min_data_in_leaf, val_score: 5486.889965:  80%|########  | 4/5 [27:30<07:03, 423.56s/it][I 2020-10-26 22:05:13,594] Trial 66 finished with value: 5486.8899651782485 and parameters: {'min_child_samples': 10}. Best is trial 66 with value: 5486.8899651782485.


min_data_in_leaf, val_score: 5486.889965:  80%|########  | 4/5 [27:30<07:03, 423.56s/it]

Training until validation scores don't improve for 10 rounds.
[50]	valid_0's rmse: 5877.79
[100]	valid_0's rmse: 5673.34
[150]	valid_0's rmse: 5598.13
[200]	valid_0's rmse: 5564.98
[250]	valid_0's rmse: 5548.34
[300]	valid_0's rmse: 5535.08
[350]	valid_0's rmse: 5526.29
Early stopping, best iteration is:
[377]	valid_0's rmse: 5523.94




min_data_in_leaf, val_score: 5486.889965:  80%|########  | 4/5 [34:54<07:03, 423.56s/it]

min_data_in_leaf, val_score: 5486.889965: 100%|##########| 5/5 [34:54<00:00, 418.90s/it]


"\n###実際にチューニングしてくれているパラメータ###\n\nparam = {\n        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),\n        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),\n        'num_leaves': trial.suggest_int('num_leaves', 2, 256),\n        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),\n        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),\n        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),\n        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),\n    }\n"

In [ ]:
best.params

NameError: ignored

lightgbm/  MANIFEST.in  README.rst  setup.py


In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM
%cd /content/LightGBM/
!mkdir build
!cmake -DUSE_GPU=1 #avoid ..
!make -j$(nproc)
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
%cd /content/LightGBM/python-package


Cloning into 'LightGBM'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 19935 (delta 10), reused 2 (delta 0), pack-reused 19909
Receiving objects: 100% (19935/19935), 15.81 MiB | 11.43 MiB/s, done.
Resolving deltas: 100% (14538/14538), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Cloning into '/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 8.51 MiB | 8.28 MiB/s, done.
Resolving deltas: 100% (17565/17565), done.
Submodule path 'compute': checked out '36c89134d4013b2e5e45bc55656a18bd6141995a'
[Errno 2] No such file or directory: '/content/LightGBM/'
/
CMake Error: The source directory "/" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on th

In [ ]:
!sudo python setup.py install --precompile

running install
running build
running build_py
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
running egg_info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'build'
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
running install_lib
Traceback (most recent call last):
  File "setup.py", line 321, in <module>
    'Topic :: Scientific/Engineering :: Artificial Intelligence'])
  File "/usr/local/lib/python3.6/dist-packages/setuptools/__init__.py", line 153, in setup
    return distutils.core.setup(**attrs)
  File "/usr/lib/python3.6/distutils/core.py", line 148, in setup
    dist.run_commands()
  File "/usr/lib

In [ ]:
pip install optuna
{'feature_fraction': 0.6}
{'num_leaves': 135}

In [ ]:
y_valid[:10]

77986     15555.0
25537      8500.0
128540    12000.0
84377      3000.0
142596     8500.0
130297     3060.0
93591      9500.0
27099     10800.0
114400     2300.0
50388      4000.0
Name: price, dtype: float64